In [1]:
from pathlib import Path
import torch
import pandas as pd
import numpy as np
import torchvision
from typing import List, Dict
import os
import matplotlib.pyplot as plt

from utils import load_model
from model_builder import ImageOnlyModel
from PIL import Image
from utils import get_data_transform
from utils import evaluate_skin_tone
from utils import predict_test_set

device = "cuda" if torch.cuda.is_available() else "cpu"

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


c:\Users\ACER NITRO\anaconda3\envs\cuda\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.4 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# Load the model

In [2]:
data_transform_densenet121 = get_data_transform(backbone="densenet121")
data_transform_efficientnet_b5 = get_data_transform(backbone="efficientnet_b5")
data_transform_inception_v3= get_data_transform(backbone="inception_v3")


In [3]:
test_dir = "models"
model_name1 = "densenet121_N_Epochs=40_batch=8_lr=0.0001.pth"
model_name2 = "efficientnet_b5_N_Epochs=40_batch=8_lr=0.0001.pth"
model_name3 = "inception_v3_N_Epochs=40_batch=8_lr=0.0001.pth"



densenet121_model = load_model(model_class=ImageOnlyModel,
                          target_dir=test_dir,
                          model_name=model_name1,
                          backbone="densenet121",
                          device=device,
                          num_classes=2)

efficientnet_b5_model = load_model(model_class=ImageOnlyModel,
                          target_dir=test_dir,
                          model_name=model_name2,
                          backbone="efficientnet_b5",
                          device=device,
                          num_classes=2)


inception_v3_model = load_model(model_class=ImageOnlyModel,
                          target_dir=test_dir,
                          model_name=model_name3,
                          backbone="inception_v3",
                          device=device,
                          num_classes=2)




c:\Users\ACER NITRO\Desktop\SID\Skin Cancer Classification\Image Only Models\utils.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.lo

[INFO] Loaded model from: models\densenet121_N_Epochs=40_batch=8_lr=0.0001.pth
[INFO] Loaded model from: models\efficientnet_b5_N_Epochs=40_batch=8_lr=0.0001.pth
[INFO] Loaded model from: models\inception_v3_N_Epochs=40_batch=8_lr=0.0001.pth


# Make prediction 

In [4]:
images_path = Path("data/")
images_path / "test"

test_metadata_path = r'data\test_data.csv'

test_metadata = pd.read_csv(test_metadata_path)
test_metadata.head()


,DDI_ID,DDI_file,malignant,DDI_path,skin_tone_12,skin_tone_34,skin_tone_56,Disease_Group_Non_melanoma,Disease_Group_melanoma
0,300,000300.png,1,data\test\malignant\000300.png,1,0,0,1,0
1,40,000040.png,1,data\test\malignant\000040.png,1,0,0,1,0
2,422,000422.png,0,data\test\benign\000422.png,1,0,0,1,0
3,289,000289.png,1,data\test\malignant\000289.png,1,0,0,1,0
4,258,000258.png,0,data\test\benign\000258.png,1,0,0,1,0


In [5]:
skin_tone_12_df = test_metadata[test_metadata["skin_tone_12"] == 1]
skin_tone_34_df = test_metadata[test_metadata["skin_tone_34"] == 1]
skin_tone_56_df = test_metadata[test_metadata["skin_tone_56"] == 1]


In [7]:
df = pd.DataFrame(columns=["tta_augmenation", "model_name", "auc", "auc_12", "auc_34", "auc_56", "accuracy", "accuracy_12", "accuracy_34", "accuracy_56"])


### Prediction With test-Time augmentation

In [9]:
true_labels_tta, predicted_classes_tta, metrics_tta = evaluate_skin_tone(model=efficientnet_b5_model, dataset=test_metadata,transform = data_transform_efficientnet_b5, device = device)
true_labels_tta_12, predicted_classes_tta_12, metrics_tta_12 = evaluate_skin_tone(model=efficientnet_b5_model, dataset=skin_tone_12_df,transform = data_transform_efficientnet_b5, device = device)
true_labels_tta_34, predicted_classes_tta_34, metrics_tta_34= evaluate_skin_tone(model=efficientnet_b5_model, dataset=skin_tone_34_df,transform = data_transform_efficientnet_b5, device = device)
true_labels_tta_56, predicted_classes_tta_56, metrics_tta_56 = evaluate_skin_tone(model=efficientnet_b5_model, dataset=skin_tone_56_df,transform = data_transform_efficientnet_b5, device = device)


df.loc[0] = [
    1,
    "efficientnet_b5",
    metrics_tta['auc'], metrics_tta_12['auc'], metrics_tta_34['auc'], metrics_tta_56['auc'],
    metrics_tta['accuracy'], metrics_tta_12['accuracy'], metrics_tta_34['accuracy'], metrics_tta_56['accuracy']
]


In [10]:
df


,tta_augmenation,model_name,auc,auc_12,auc_34,auc_56,accuracy,accuracy_12,accuracy_34,accuracy_56
0,1,efficientnet_b5,0.825,0.752941,0.848958,0.705882,0.863636,0.772727,0.954545,0.863636


In [12]:
true_labels_tta, predicted_classes_tta, metrics_tta = evaluate_skin_tone(model=inception_v3_model, dataset=test_metadata,transform = data_transform_inception_v3, device = device)
true_labels_tta_12, predicted_classes_tta_12, metrics_tta_12 = evaluate_skin_tone(model=inception_v3_model, dataset=skin_tone_12_df,transform = data_transform_inception_v3, device = device)
true_labels_tta_34, predicted_classes_tta_34, metrics_tta_34= evaluate_skin_tone(model=inception_v3_model, dataset=skin_tone_34_df,transform = data_transform_inception_v3, device = device)
true_labels_tta_56, predicted_classes_tta_56, metrics_tta_56 = evaluate_skin_tone(model=inception_v3_model, dataset=skin_tone_56_df,transform = data_transform_inception_v3, device = device)


df.loc[1] = [
    1,
    "inception_v3",
    metrics_tta['auc'], metrics_tta_12['auc'], metrics_tta_34['auc'], metrics_tta_56['auc'],
    metrics_tta['accuracy'], metrics_tta_12['accuracy'], metrics_tta_34['accuracy'], metrics_tta_56['accuracy']
]


In [14]:
true_labels_tta, predicted_classes_tta, metrics_tta = evaluate_skin_tone(model=densenet121_model, dataset=test_metadata,transform = data_transform_densenet121, device = device)
true_labels_tta_12, predicted_classes_tta_12,  metrics_tta_12 = evaluate_skin_tone(model=densenet121_model, dataset=skin_tone_12_df,transform = data_transform_densenet121, device = device)
true_labels_tta_34, predicted_classes_tta_34,  metrics_tta_34= evaluate_skin_tone(model=densenet121_model, dataset=skin_tone_34_df,transform = data_transform_densenet121, device = device)
true_labels_tta_56, predicted_classes_tta_56,  metrics_tta_56 = evaluate_skin_tone(model=densenet121_model, dataset=skin_tone_56_df,transform = data_transform_densenet121, device = device)


df.loc[2] = [
    1,
    "densenet121",
    metrics_tta['auc'], metrics_tta_12['auc'], metrics_tta_34['auc'], metrics_tta_56['auc'],
    metrics_tta['accuracy'], metrics_tta_12['accuracy'], metrics_tta_34['accuracy'], metrics_tta_56['accuracy']
]


In [16]:
indexes = df.index
Gap = []
for index in indexes:
    auc_12, auc_34, auc_56 = df.loc[index, "auc_12"], df.loc[index, "auc_34"], df.loc[index, "auc_34"]
    AUC_max,  AUC_min = np.max([auc_12, auc_34, auc_56]), np.min([auc_12, auc_34, auc_56])
    gap = AUC_max - AUC_min
    Gap.append(gap)

df["Gap"] = Gap


In [17]:
df.sort_values(by="Gap", ascending=True) 


,tta_augmenation,model_name,auc,auc_12,auc_34,auc_56,accuracy,accuracy_12,accuracy_34,accuracy_56,Gap
1,1,inception_v3,0.78125,0.811765,0.729167,0.764706,0.803030,0.818182,0.636364,0.909091,0.082598
0,1,efficientnet_b5,0.82500,0.752941,0.848958,0.705882,0.863636,0.772727,0.954545,0.863636,0.096017
2,1,densenet121,0.80750,0.705882,0.895833,0.805882,0.772727,0.636364,0.818182,0.863636,0.189951


In [19]:
df.sort_values(by="auc", ascending=False) 


,tta_augmenation,model_name,auc,auc_12,auc_34,auc_56,accuracy,accuracy_12,accuracy_34,accuracy_56,Gap
0,1,efficientnet_b5,0.82500,0.752941,0.848958,0.705882,0.863636,0.772727,0.954545,0.863636,0.096017
2,1,densenet121,0.80750,0.705882,0.895833,0.805882,0.772727,0.636364,0.818182,0.863636,0.189951
1,1,inception_v3,0.78125,0.811765,0.729167,0.764706,0.803030,0.818182,0.636364,0.909091,0.082598
